# 환경구축 및 최초 실행(머신 시작시마다 실행)

In [3]:
# 필요한 파일 설치
!apt -q update
!apt -yq install python3.10
!apt -yq install libpython3.10-dev build-essential aria2
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
!python3.10 -m pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!python3.10 -m pip install -q xformers==0.0.18 triton==2.0.0 -U
!python3.10 -m pip install -q httpx==0.24.1 -U
!python3.10 -m pip install -q matplotlib -U
!python3.10 -m pip install -q ipython -U
!python3.10 -m pip install -q pickleshare -U

import os
import glob
from IPython import get_ipython

# 그래프그리기 라이브러리를 인라인으로 표시하는 설정 
get_ipython().run_line_magic('matplotlib', 'inline')

# Stable Diffusion WebUI의 리포지티로리가 없으면 복제 
if not os.path.isdir("/notebooks/ui"):
    %cd /notebooks
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /notebooks/ui
    # Stable Diffusion WebUI の v1.6.0을 사용함 
    %cd /notebooks/ui
    !git checkout master
    !git checkout 5ef669de080814067961f28357256e8fe27544f4

# Model이 없는 경우 다운로드 
if not glob.glob("/notebooks/ui/models/Stable-diffusion/*.safetensors"):
    os.system("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v0.8.0.safetensors -d /notebooks/ui/models/Stable-diffusion -o blue_pencil-XL-v0.8.0.safetensors")

# 설정변경 
%cd /notebooks/ui
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /notebooks/ui/modules/shared_options.py

# SDXL용 UI 편집
if os.path.exists("/notebooks/ui/ui-config.json"):
    !sed -i -e 's/value": 512/value": 1024/g' /notebooks/ui/ui-config.json
    !sed -i -e 's/Grid margins (px)\/value": 0/Grid margins (px)\/value": 5/g' /notebooks/ui/ui-config.json

if os.path.exists("/notebooks/ui/config.json"):
    !sed -i -e 's/"\[date\]"/"\[date\]-\[model_name\]"/g' /notebooks/ui/config.json

# Stable Diffusion WebUI 실행 
!python3.10 launch.py --xformers --enable-insecure-extension-access --share --gradio-queue --no-half-vae

Get:1 file:/var/cuda-repo-ubuntu2204-12-0-local  InRelease [1575 B]
Get:1 file:/var/cuda-repo-ubuntu2204-12-0-local  InRelease [1575 B]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://deb.nodesource.com/node_20.x nodistro InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Reading package lists...
Building dependency tree...
Reading state information...
184 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the

# ⭐ 최초 시작 이외의 실행

In [2]:
# SDXL용 UI편집
if os.path.exists("/notebooks/ui/ui-config.json"):
    !sed -i -e 's/value": 512/value": 1024/g' /notebooks/ui/ui-config.json
    !sed -i -e 's/Grid margins (px)\/value": 0/Grid margins (px)\/value": 5/g' /notebooks/ui/ui-config.json
    
if os.path.exists("/notebooks/ui/config.json"):
    !sed -i -e 's/"\[date\]"/"\[date\]-\[model_name\]"/g' /notebooks/ui/config.json
    
# Stable Diffusion WebUI 실행 
%cd /notebooks/ui
!python3.10 launch.py --xformers --enable-insecure-extension-access --share --gradio-queue --no-half-vae --skip-version-check

/notebooks/ui
Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Version: v1.6.0
Commit hash: 5ef669de080814067961f28357256e8fe27544f4
Traceback (most recent call last):
  File "/notebooks/ui/launch.py", line 48, in <module>
    main()
  File "/notebooks/ui/launch.py", line 39, in main
    prepare_environment()
  File "/notebooks/ui/modules/launch_utils.py", line 356, in prepare_environment
    raise RuntimeError(
RuntimeError: Torch is not able to use GPU; add --skip-torch-cuda-test to COMMANDLINE_ARGS variable to disable this check


# Model 다운로드 

In [ ]:
"""
다운로드하지 않는 것은 주석처리한다.
주석처리 방법: #을 사용하여 주석처리한다.
"""

# 다운로드 방법 정의 
ARIA2="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# Model 디렉토리 정의 
MODEL_DIR="/notebooks/ui/models/Stable-diffusion"
!rm -rf {MODEL_DIR}/* #학습모델 모두 삭제(리셋) 함 

# blue_pencil　
!{ARIA2} https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v0.8.0.safetensors -d {MODEL_DIR} -o blue_pencil-XL-v0.8.0.safetensors

# sdvn7Nijistylexl_v1
# !{ARIA2} https://civitai.com/api/download/models/155870 -d {MODEL_DIR} -o sdvn7Nijistylexl_v1.safetensors

# CherryPickerXL26　
# !{ARIA2} https://civitai.com/api/download/models/173768 -d {MODEL_DIR} -o CherryPickerXL26.safetensors

# Refiner
# !{ARIA2} https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -d {MODEL_DIR} -o sd_xl_refiner_1.0.safetensors

# LoRA, Embedings 다운로드 

In [ ]:
"""
다운로드하지 않는 것은 주석처리한다.
주석처리 방법: #을 사용하여 주석처리한다.
"""

# 다운로드 방법 정의 
ARIA2="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# LoRA 디렉토리 정의 
LORA_DIR="/notebooks/ui/models/Lora" 
# Embedings 디렉토리 정의 
EMBEDDINGS_DIR="/notebooks/ui/embeddings"


# LoRA 다운로드 ######################################################################
!rm -rf {LORA_DIR}/*  #LoRA를 모두 삭제(리셋) 함
!{ARIA2} https://civitai.com/api/download/models/155932 -d {LORA_DIR} -o ClearHand-V2.safetensors
# !{ARIA2} https://civitai.com/api/download/models/62833 -d {LORA_DIR} -o detail-tweaker-xl.safetensors


# Embedings 다운로드 ######################################################################
!rm -rf {EMBEDDINGS_DIR}/*  #Embedings를 모두 삭제(리셋) 함
!{ARIA2} https://civitai.com/api/download/models/175819 -d {EMBEDDINGS_DIR} -o unaestheticXL_Sky3.1.safetensors
!{ARIA2} https://civitai.com/api/download/models/162146 -d {EMBEDDINGS_DIR} -o unaestheticXL_AYv1.safetensors
!{ARIA2} https://civitai.com/api/download/models/149308 -d {EMBEDDINGS_DIR} -o unaestheticXLv31.safetensors

# 확장 프로그램 설치

In [ ]:
"""
다운로드하지 않는 것은 주석처리한다.
주석처리 방법: #을 사용하여 주석처리한다.
"""

# 확장기능 디렉토리 정의
EXTENSIONS_DIR = "/notebooks/ui/extensions"

# 확장프로그램 설치 
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {EXTENSIONS_DIR}/images-browser
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing {EXTENSIONS_DIR}/infinite-image-browsing
!git clone https://github.com/Haoming02/sd-webui-prompt-format {EXTENSIONS_DIR}/prompt-format
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {EXTENSIONS_DIR}/tagcomplete
!git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger {EXTENSIONS_DIR}/wd14-tagger
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one {EXTENSIONS_DIR}/prompt-all-in-one
!git clone https://github.com/Bing-su/adetailer {EXTENSIONS_DIR}/adetailer
!git clone https://github.com/Mikubill/sd-webui-controlnet {EXTENSIONS_DIR}/sd-webui-controlnet
!git clone https://github.com/anonCantCode/sd-webui-send-to-controlnet {EXTENSIONS_DIR}/send-to-controlnet
!git clone https://github.com/jexom/sd-webui-depth-lib {EXTENSIONS_DIR}/depth-lib
!git clone https://github.com/IDEA-Research/DWPose {EXTENSIONS_DIR}/DWPose
!git clone https://github.com/huchenlei/sd-webui-openpose-editor {EXTENSIONS_DIR}/openpose-editor
!git clone https://github.com/ljleb/sd-webui-freeu {EXTENSIONS_DIR}/freeu
!git clone https://github.com/alemelis/sd-webui-ar {EXTENSIONS_DIR}/sd-webui-ar
!git clone https://github.com/yankooliveira/sd-webui-photopea-embed  {EXTENSIONS_DIR}/photopea-embed
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg {EXTENSIONS_DIR}/rembg
# !git clone https://github.com/aka7774/sd_katanuki {EXTENSIONS_DIR}/sd_katanuki
# !git clone https://github.com/AlUlkesh/sd_save_intermediate_images {EXTENSIONS_DIR}/sd_save_intermediate_images

# wd14-tagger 오류 대책（커밋 → 99bf7d8132cb757edbcdfaad654f31f32f3500a5）
!sed -i -e 's/from webui import wrap_gradio_gpu_call/from modules.call_queue import wrap_gradio_gpu_call/g' /notebooks/ui/extensions/wd14-tagger/tagger/ui.py
!sed -i -e 's/from modules.shared import models_path/import argparse\nimport os\n\nmodules_path = os.path.dirname(os.path.realpath(__file__))\n\nparser_pre = argparse.ArgumentParser(add_help=False)\nparser_pre.add_argument("--data-dir", type=str, default=os.path.dirname(modules_path), help="base path where all user data is stored", )\ncmd_opts_pre = parser_pre.parse_known_args()[0]\ndata_path = cmd_opts_pre.data_dir\n\nmodels_path = os.path.join(data_path, "models")/g' /notebooks/ui/extensions/wd14-tagger/preload.py

# sd-webui-ar 확장에 SDXL권장 크기 쓰기
f = open("/notebooks/ui/extensions/sd-webui-ar/resolutions.txt", "w")
text = """SDv2, 512, 512 # SDXL以前のサイズ
SDXL 1:1, 1024, 1024 # Square
SDXL 2:3, 836, 1254 # Landscape
SDXL 3:4, 886, 1182 # Photo
SDXL 9:16, 768, 1344 # Vertical"""
f.write(text)
f.close()

# SDXL용 ControlNet Model, ADetailer Model 다운로드 

In [ ]:
"""
다운로드하지 않는 것은 주석처리한다.
주석처리 방법: #을 사용하여 주석처리한다.
"""

# 다운로드 방법 정의 
ARIA2="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# Hugging Face의 ControlNet Model 공통 다운로드 URL 정의함 
HF_CTN_URL="https://huggingface.co/lllyasviel/sd_control_collection/resolve/main"


# ControlNet Model 다운로드 ---------------------------------------------------
CTN_MODEL_DIR = "/notebooks/ui/models/ControlNet"
!rm -rf {CTN_MODEL_DIR}/* #ControlNet Model 모두 삭제 (리셋)

# Canny
# !{ARIA2} {HF_CTN_URL}/sai_xl_canny_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_canny_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_canny_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_canny_128lora.safetensors #396 MB
# !{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_canny.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_canny.safetensors #46.2 MB

# Depth
# !{ARIA2} {HF_CTN_URL}/sai_xl_depth_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_depth_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_depth_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_depth_128lora.safetensors #396 MB
!{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_depth.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_depth.safetensors #46.2 MB

# Openpose
# !{ARIA2} {HF_CTN_URL}/thibaud_xl_openpose_256lora.safetensors -d {CTN_MODEL_DIR} -o thibaud_xl_openpose_256lora.safetensors #774 MB
!{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_openpose_anime_v2.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_openpose_anime_v2.safetensors #46.2 MB

# Lineart
!{ARIA2} {HF_CTN_URL}/t2i-adapter_diffusers_xl_lineart.safetensors -d {CTN_MODEL_DIR} -o t2i-adapter_diffusers_xl_lineart.safetensors #158 MB

# Softedge
# !{ARIA2} {HF_CTN_URL}/sargezt_xl_softedge.safetensors -d {CTN_MODEL_DIR} -o sargezt_xl_softedge.safetensors #2.5 GB

# Scribble
# !{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_scribble_anime.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_scribble_anime.safetensors #46.2 MB

# Sketch
# !{ARIA2} {HF_CTN_URL}/sai_xl_sketch_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_sketch_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_sketch_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_sketch_128lora.safetensors #396 MB

# Segmentaion
# !{ARIA2} https://civitai.com/api/download/models/154866 -d {CTN_MODEL_DIR} -o Segmentaion.safetensors #84.3 MB

# Recolor
# !{ARIA2} {HF_CTN_URL}/sai_xl_recolor_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_recolor_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_recolor_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_recolor_128lora.safetensors #396 MB

# ip-adapter
# !{ARIA2} {HF_CTN_URL}/ip-adapter_xl.pth -d {CTN_MODEL_DIR} -o ip-adapter_xl.pth #703 MB
!{ARIA2} https://civitai.com/api/download/models/155532 -d {CTN_MODEL_DIR} -o h94-ip-adapter-xl.pth #670 MB

# Blur
!{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_blur.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_blur.safetensors #46.2 MB
# !{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_blur_anime.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_blur_anime.safetensors #46.2 MB


# ip-adapter, revision 다운로드 파일 ---------------------------------------------------
CTN_DOWNLOADS = "/notebooks/ui/extensions/sd-webui-controlnet/annotator/downloads"
!rm -fr {CTN_DOWNLOADS}/* #ControlNet 모든 다운로드 파일 삭제 (리셋)
# !{ARIA2} https://huggingface.co/lllyasviel/Annotators/resolve/main/clip_g.pth -d {CTN_DOWNLOADS}/clip_vision -o clip_g.pth #3.44 GB


# ADetailer Model 추가 폴더  ---------------------------------------------------
ADETAILER_MODEL_DIR = "/notebooks/ui/models/adetailer"
!rm -fr {ADETAILER_MODEL_DIR}/* #ADetailer Model 모두 삭제 (리셋)
!{ARIA2} https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt -d {ADETAILER_MODEL_DIR} -o hand_yolov8s.pt

# 이미지 생성결과를 ZIP으로 만들기 

In [ ]:
"""
이미지 생성결과를 7ZIP 형식으로 압축한다. 
압축된 파일은 업데이트 버튼을 누르면 표시된다. 
이미지 생성된 파일은 'bak'폴더로 이동된다. 
"""

%cd /notebooks
!apt install -y p7zip-full

import os
import glob
import datetime
import shutil

# 오늘일자 받기 
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 압축할 이미지 폴더명 
folders = ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]

# 이미지 폴더를 이동하고 압축하는 기능 
def compress_images(folders):
    # 이동 경로 
    destination = f"/notebooks/bak/outputs-{today}"
    # 압축대상경로 
    archive = f"outputs-{today}.7z"
    # 이동대상폴더 생성 
    os.makedirs(destination, exist_ok=True)
    # 각 폴더에 대해 처리 
    for folder in folders:
        # 이동경로 
        source = f"/notebooks/ui/outputs/{folder}"
        source_folder = f"{destination}/{folder}"
        # 이동경로에 이미지가 있으면 
        if os.listdir(source):
            os.makedirs(source_folder, exist_ok=True)
            # 이미지 이동위치로 이동 
            for file in glob.glob(f"{source}/*"):
                shutil.move(file, source_folder)
            print("⭐-------------------------------------------⭐\n  실행 → ", source)
    # 이동 대상 폴더 압축 
    !7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{archive}" "{destination}"

# 함수호출 
compress_images(folders)

# AUTOMATIC1111업데이트（업데이트가 필요할 때만 실행）

In [ ]:
# Stable Diffusion WebUI을 최신버전으로 업데이트 함 
%cd /notebooks/ui
!git checkout master
!git pull

#실패시 
# !git pull https://github.com/AUTOMATIC1111/stable-diffusion-webui master

# 용량확인 및 정리 

In [ ]:
"""
삭제하지 않는 것은 주석처리한다.
주석처리 방법: #을 사용하여 주석처리한다.
"""
# 10MB이상의 파일위치와 크기를 내림차순으로 나열함 
%cd /notebooks
!find . -size +10M -print0 | du -ch --files0-from=- | sort -hr

# bak 내용 모두 지우기  ---------------------------------------------------
!rm -fr ./bak/*

# 실행시 자동 다운로드되는 파일 삭제하기  ---------------------------------------------------
!rm -fr ./ui/extensions/sd-webui-controlnet/annotator/downloads/* 
!rm -f ./ui/models/BLIP/model_base_caption_capfilt_large.pth
!rm -f ./ui/models/torch_deepdanbooru/model-resnet_custom_v3.pt

# 각 모델파일 모두 지우기  ---------------------------------------------------
# !rm -f ./ui/models/Stable-diffusion/* # Model
# !rm -f ./ui/models/Lora/* # LoRA
# !rm -f ./ui/models/ControlNet/* # Controlnet

# 대상파일을 삭제하고 싶은 경우  ---------------------------------------------------
# !rm -f <삭제할 파일 경로>